In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

# Choose a name for your CPU cluster
compute_cluster_name = "compute-cluster"

# Verify that the cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           idle_seconds_before_scaledown=2400,
                                                           min_nodes=0,
                                                           max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os #> es wird in Bild 4 erwähnt

# Specify parameter sampler
#ps = ### YOUR CODE HERE ### -> uniform. 'Tune hyperparmaters for your model with Azure ML'
ps = RandomParameterSampling( {
        "learning_rate": uniform(0.01, 4),
        "batch_size": uniform(10, 300)
    }
)

# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor = 0.1) # early_termination_policy

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ### 'SKLearn Class' 
est = SKLearn(source_directory='.', #kann vielleicht entfallen
              compute_target=compute_cluster,
              script_params={'--C': 1.0, '--max_iter': 100}, #alternatively: ['--C', 1.0, '--max_iter', 100]
              entry_script='train.py') #A string representing the relative path to the file used to start training.

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ### 'Tune hyperparameters for your model with Azure ML'
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4) #similar to the number of nodes

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=False)
RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()

joblib.dump(best_run, 'SKlearn_hyperdrive.joblib')

# "Train image classification model with..."

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
dataset = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv', separator=',', header=True, validate=False)
#ds = dataset.to_pandas_dataframe() 


In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(dataset)
#or maybe without the to_pandas_dataframe()

#todo: Split data into train and test set
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification', #logisticRegression is a classification task
    primary_metric='accuracy',
    training_data=x_train, 
    label_column_name=y_train,
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

run = exp.submit(automl_config, show_output=True)
run.wait_for_completion(show_output=False)
RunDetails(run).show()

In [1]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_automl_model = run.get_best_run_by_primary_metric()

joblib.dump(best_automl_model, 'AutoML.joblib')

NameError: name 'run' is not defined

In [ ]:
print(finished)